#  Usage/Benchmark Tests
-------------------


In [1]:
# Load Params 
using BenchmarkTools
using Quack;
const Qtrees = Quack;

┌ Info: Precompiling Quack [332d158b-1d50-4650-beb3-f7fe5de64c74]
└ @ Base loading.jl:1260


In [2]:
const QT_TEST_SIZE, RM_TEST_SIZE = 100_000, 10_000

## Inserts - Tree Build
ps = Qtrees.genRandomCoords(0., 1., 0., 1., QT_TEST_SIZE);

sampleQTree = Qtrees.qtBox(Qtrees.Coord(.0, .0), 1.)

@time (for p in ps Qtrees.insertIntoQuadTree!(sampleQTree, p) end)

  0.267455 seconds (2.27 M allocations: 83.137 MiB, 42.29% gc time)


In [3]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")

Total Obj Size: 8.812488 MB


In [4]:
@benchmark Qtrees.queryRange(sampleQTree, Qtrees.Box(Qtrees.Coord(rand() , rand()), rand()/2))

BenchmarkTools.Trial: 
  memory estimate:  6.84 KiB
  allocs estimate:  117
  --------------
  minimum time:     6.332 μs (0.00% GC)
  median time:      891.547 μs (0.00% GC)
  mean time:        1.565 ms (9.90% GC)
  maximum time:     15.501 ms (41.15% GC)
  --------------
  samples:          3177
  evals/sample:     1

In [10]:
#Test in kilometers, 5km ~0.045 on a flat plane
@benchmark Qtrees.radialSearch(sampleQTree, Qtrees.Coord(rand(), rand()), 5.)

MethodError: MethodError: no method matching haversineDistance(::Base.UUID, ::Quack.Coord)
Closest candidates are:
  haversineDistance(!Matched::Quack.Coord, ::Quack.Coord) at /Users/dustinwilson/Desktop/Quack/src/Quack.jl:265

In [6]:
# Oyy...function modifies the structure, use @time instead of benchmark
@time (for i in 1:RM_TEST_SIZE Qtrees.removePoint!(sampleQTree, ps[rand(1:QT_TEST_SIZE)]) end)

  0.104252 seconds (666.75 k allocations: 31.680 MiB)


In [7]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(Qtrees.queryRange(sampleQTree, Qtrees.Box(Qtrees.Coord(0. , 0.), 1))))")

Total Obj Size: 8.5074 MB
Total Point Count: 90466


In [8]:
@time Qtrees.cleanUpQTree(sampleQTree)

  0.040959 seconds (115.63 k allocations: 4.895 MiB)


true

In [9]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(Qtrees.queryRange(sampleQTree, Qtrees.Box(Qtrees.Coord(0. , 0.), 1))))")

Total Obj Size: 8.421136 MB
Total Point Count: 90466
